# 4-1 张量的结构操作

张量的操作主要包括：
* 张量的结构操作
* 张量的数学运算

张量结构操作诸如：
* 张量创建
* 索引切片
* 维度变换
* 合并分割

张量数学运算主要有：
* 标量运算
* 向量运算
* 矩阵运算
* 广播机制

## 一、创建张量
张量的创建方法中有很多和numpy中创建array的方法很像

In [1]:
import numpy as np
import torch

In [2]:
a = torch.tensor([1, 2, 3], dtype=torch.float)
print(a)

tensor([1., 2., 3.])


In [3]:
b = torch.arange(1, 10, step=2)
b

tensor([1, 3, 5, 7, 9])

In [4]:
c = torch.linspace(0.0, 10, 10)
c

tensor([ 0.0000,  1.1111,  2.2222,  3.3333,  4.4444,  5.5556,  6.6667,  7.7778,
         8.8889, 10.0000])

In [5]:
d = torch.zeros([3, 3])
print(d)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [6]:
a = torch.ones((3, 3), dtype=torch.int)
b = torch.zeros_like(a, dtype=torch.int)
print(b)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]], dtype=torch.int32)


In [7]:
torch.fill_(b, 5)

tensor([[5, 5, 5],
        [5, 5, 5],
        [5, 5, 5]], dtype=torch.int32)

In [8]:
# 均匀随机分布
torch.manual_seed(0)
minval, maxval = 0, 10
a = minval + (maxval-minval)*torch.rand([5])
a

tensor([4.9626, 7.6822, 0.8848, 1.3203, 3.0742])

In [9]:
# 正态分布随机
b = torch.normal(torch.zeros(3, 3), std=torch.ones(3, 3))
b

tensor([[ 0.5507,  0.2704,  0.6472],
        [ 0.2490, -0.3354,  0.4564],
        [-0.6255,  0.4539, -1.3740]])

In [10]:
# 正态随机分布
mean, std = 2, 5
c = std*torch.randn([3, 3]) + mean
c

tensor([[16.2371, -1.6612,  3.9163],
        [ 7.4999,  1.5616,  4.0768],
        [ 5.2128, -8.9407,  6.4601]])

In [11]:
# 整数随机排列
d = torch.randperm(5)
d

tensor([3, 1, 0, 4, 2])

In [12]:
# 特殊矩阵
I = torch.eye(3, 3)  # 单位矩阵
print(I)
t = torch.diag(torch.tensor([1, 2, 3]))  # 对角矩阵
print(t)

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])
tensor([[1, 0, 0],
        [0, 2, 0],
        [0, 0, 3]])


## 二、索引切片
张量的索引切片方式和numpy几乎一样。切片时, 支持缺省参数和省略号。

可以通过索引和切片对部分元素进行修改。

**此外, 对于不规则的切片提取, 可以使用torch.index_select, torch.masked_select, torch.take**

**如果想要通过修改张量的某些元素得到新的张量, 可以使用torch.where, torch.masked_fill, torch.index_fill**

In [13]:
# 均匀随机分布
torch.manual_seed(0)
minval, maxval = 0, 10
t = torch.floor(minval + (maxval-minval)*torch.rand([5, 5])).int()
print(t)

tensor([[4, 7, 0, 1, 3],
        [6, 4, 8, 4, 6],
        [3, 4, 0, 1, 2],
        [5, 6, 8, 1, 2],
        [6, 9, 3, 8, 4]], dtype=torch.int32)


In [14]:
# 第0行
print(t[[0], :])

tensor([[4, 7, 0, 1, 3]], dtype=torch.int32)


In [15]:
# 倒数第一行
print(t[-1])

tensor([6, 9, 3, 8, 4], dtype=torch.int32)


In [16]:
# 第1行第3列
print(t[1, 3])
print(t[1][3])

tensor(4, dtype=torch.int32)
tensor(4, dtype=torch.int32)


In [17]:
# 第1行至第3行
print(t[1:4, :])

tensor([[6, 4, 8, 4, 6],
        [3, 4, 0, 1, 2],
        [5, 6, 8, 1, 2]], dtype=torch.int32)


In [18]:
# 第1行至最后一行, 第0行到最后一列每个两列
print(t[1:4, :4:2])

tensor([[6, 8],
        [3, 0],
        [5, 8]], dtype=torch.int32)


In [19]:
# 可以使用索引和切片修改部分元素
x = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32, requires_grad=True)
x.detach()[1, :] = torch.tensor([0.0, 0.0])
x

tensor([[1., 2.],
        [0., 0.]], requires_grad=True)

In [20]:
a = torch.arange(27).view(3, 3, 3)
a

tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8]],

        [[ 9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]])

In [22]:
# 省略号可以表示多个冒号
print(a[..., 1])

tensor([[ 1,  4,  7],
        [10, 13, 16],
        [19, 22, 25]])


以上的切片方式相对规则, 对于不规则的切片提取, 可以使用torch.index_select, torch.take, torch.gather, torch.masked_select。

考虑班级成绩册的例子, 有4个班级, 每个班级有10个学生, 每个学生7门科目。可以用一个4*10*7的张量来表示。

In [23]:
minval = 0
maxval = 100
scores = torch.floor(minval + (maxval-minval)*torch.rand([4, 10, 7])).int()
scores

tensor([[[55, 95,  3, 18, 37, 30, 93],
         [17, 26, 15,  3, 20, 92, 72],
         [74, 52, 24, 58,  3, 13, 24],
         [81, 79, 27, 48, 81, 99, 69],
         [56, 83, 20, 59, 11, 15, 24],
         [72, 70, 20, 65, 77, 43, 51],
         [61, 81, 98, 11, 31, 69, 91],
         [93, 94, 59,  6, 54, 18,  3],
         [94, 88,  0, 59, 41, 41, 27],
         [69, 20, 68, 75, 85, 68,  0]],

        [[17, 74, 60, 10, 21, 97, 83],
         [28, 37,  2, 49, 12, 11, 47],
         [57, 29, 79, 19, 95, 84,  7],
         [37, 52, 57, 61, 69, 52, 25],
         [73,  2, 20, 37, 25, 32,  9],
         [39, 60, 17, 47, 85, 44, 51],
         [45, 60, 81, 97, 81, 97, 46],
         [ 5, 26, 84, 49, 25, 11,  3],
         [ 7, 39, 77, 77,  1, 81, 10],
         [39, 29, 40, 40,  5,  6, 42]],

        [[50, 27, 68,  4, 46, 93, 29],
         [95, 68,  4, 81, 44, 27, 89],
         [ 9, 55, 39, 85, 63, 74, 67],
         [37, 39,  8, 77, 89, 84, 14],
         [52, 14, 22, 20, 67, 20, 48],
         [52, 82, 12,

In [27]:
# 抽取每个班级第0个学生, 第5个学生, 第9个学生的全部成绩
sel = torch.index_select(scores, dim=1, index=torch.tensor([0, 5, 9]))
print(sel.shape)

torch.Size([4, 3, 7])


In [28]:
# 抽取每个班级第0个学生, 第5个学生, 第9个学生的第1门课程, 第3门课程, 第6门课程
q = torch.index_select(torch.index_select(scores, dim=1, index=torch.tensor([0, 5, 9])), dim=2, index=torch.tensor([1, 3, 6]))
print(q)
print(q.shape)

tensor([[[95, 18, 93],
         [70, 65, 51],
         [20, 75,  0]],

        [[74, 10, 83],
         [60, 47, 51],
         [29, 40, 42]],

        [[27,  4, 29],
         [82, 15, 32],
         [42, 91, 80]],

        [[72, 28, 87],
         [26, 60, 93],
         [33, 58, 59]]], dtype=torch.int32)
torch.Size([4, 3, 3])


In [29]:
# 抽取第0个班级第0个学生的第0门课程, 第2个班级的第4个学生的第1门课程, 第3个班级的第9个学生第6门课程
# take将输入看做是一个一维数组, 输出和index同形状
s = torch.take(scores, torch.tensor([0*0*7+0, 2*10*7+4*7+1, 3*10*7+9*7+6]))
s

tensor([55, 14, 59], dtype=torch.int32)

In [30]:
# 抽取分数大于80分的分数
g = torch.masked_select(scores, scores>=80)
g

tensor([95, 93, 92, 81, 81, 99, 83, 81, 98, 91, 93, 94, 94, 88, 85, 97, 83, 95,
        84, 85, 81, 97, 81, 97, 84, 81, 93, 95, 81, 89, 85, 89, 84, 82, 84, 92,
        90, 99, 91, 94, 80, 87, 86, 91, 88, 91, 93, 89, 91, 84],
       dtype=torch.int32)

以上这些方法仅能提取张量的部分元素, 但不能更改张量的部分元素值得到新的张量。

如果要通过修改张量的部分元素值得到新的张量, 可以使用torch.where, torch.index_fill和torch.masked_fill

torch.where可以理解为if的张量版本

torch.index_fill的选取元素逻辑和torch.index_select相同

torch.masked_fill的选取元素逻辑和torch.masked_select相同

In [31]:
# 如果分数大于60, 赋值为1, 否则赋值为0
ifpass = torch.where(scores>60, torch.tensor(1), torch.tensor(0))
print(ifpass)

tensor([[[0, 1, 0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 1, 1],
         [1, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 1, 1, 1],
         [0, 1, 0, 0, 0, 0, 0],
         [1, 1, 0, 1, 1, 0, 0],
         [1, 1, 1, 0, 0, 1, 1],
         [1, 1, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0],
         [1, 0, 1, 1, 1, 1, 0]],

        [[0, 1, 0, 0, 0, 1, 1],
         [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 1, 1, 0],
         [0, 0, 0, 1, 1, 0, 0],
         [1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0],
         [0, 0, 1, 1, 1, 1, 0],
         [0, 0, 1, 0, 0, 0, 0],
         [0, 0, 1, 1, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 1, 0, 0, 1, 0],
         [1, 1, 0, 1, 0, 0, 1],
         [0, 0, 0, 1, 1, 1, 1],
         [0, 0, 0, 1, 1, 1, 0],
         [0, 0, 0, 0, 1, 0, 0],
         [0, 1, 0, 0, 0, 1, 0],
         [1, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0],
         [1, 1, 0, 0, 1, 0, 0],
         [0, 0, 0, 1, 0, 1, 1]],

        [[0, 1, 0, 0, 1, 1, 1],
  

In [32]:
# 将每个班级第0个学生, 第5个学生, 第9个学生的全部成绩赋值为满分
torch.index_fill(scores, dim=1, index=torch.tensor([0, 5, 9]), value = 100)

tensor([[[100, 100, 100, 100, 100, 100, 100],
         [ 17,  26,  15,   3,  20,  92,  72],
         [ 74,  52,  24,  58,   3,  13,  24],
         [ 81,  79,  27,  48,  81,  99,  69],
         [ 56,  83,  20,  59,  11,  15,  24],
         [100, 100, 100, 100, 100, 100, 100],
         [ 61,  81,  98,  11,  31,  69,  91],
         [ 93,  94,  59,   6,  54,  18,   3],
         [ 94,  88,   0,  59,  41,  41,  27],
         [100, 100, 100, 100, 100, 100, 100]],

        [[100, 100, 100, 100, 100, 100, 100],
         [ 28,  37,   2,  49,  12,  11,  47],
         [ 57,  29,  79,  19,  95,  84,   7],
         [ 37,  52,  57,  61,  69,  52,  25],
         [ 73,   2,  20,  37,  25,  32,   9],
         [100, 100, 100, 100, 100, 100, 100],
         [ 45,  60,  81,  97,  81,  97,  46],
         [  5,  26,  84,  49,  25,  11,   3],
         [  7,  39,  77,  77,   1,  81,  10],
         [100, 100, 100, 100, 100, 100, 100]],

        [[100, 100, 100, 100, 100, 100, 100],
         [ 95,  68,   4,  81, 

In [33]:
b = torch.masked_fill(scores, scores<60, 60)
b

tensor([[[60, 95, 60, 60, 60, 60, 93],
         [60, 60, 60, 60, 60, 92, 72],
         [74, 60, 60, 60, 60, 60, 60],
         [81, 79, 60, 60, 81, 99, 69],
         [60, 83, 60, 60, 60, 60, 60],
         [72, 70, 60, 65, 77, 60, 60],
         [61, 81, 98, 60, 60, 69, 91],
         [93, 94, 60, 60, 60, 60, 60],
         [94, 88, 60, 60, 60, 60, 60],
         [69, 60, 68, 75, 85, 68, 60]],

        [[60, 74, 60, 60, 60, 97, 83],
         [60, 60, 60, 60, 60, 60, 60],
         [60, 60, 79, 60, 95, 84, 60],
         [60, 60, 60, 61, 69, 60, 60],
         [73, 60, 60, 60, 60, 60, 60],
         [60, 60, 60, 60, 85, 60, 60],
         [60, 60, 81, 97, 81, 97, 60],
         [60, 60, 84, 60, 60, 60, 60],
         [60, 60, 77, 77, 60, 81, 60],
         [60, 60, 60, 60, 60, 60, 60]],

        [[60, 60, 68, 60, 60, 93, 60],
         [95, 68, 60, 81, 60, 60, 89],
         [60, 60, 60, 85, 63, 74, 67],
         [60, 60, 60, 77, 89, 84, 60],
         [60, 60, 60, 60, 67, 60, 60],
         [60, 82, 60,

In [35]:
# in-place操作
scores.index_fill_(dim=1, index=torch.tensor([0, 5, 9]), value=100)
scores

tensor([[[100, 100, 100, 100, 100, 100, 100],
         [ 17,  26,  15,   3,  20,  92,  72],
         [ 74,  52,  24,  58,   3,  13,  24],
         [ 81,  79,  27,  48,  81,  99,  69],
         [ 56,  83,  20,  59,  11,  15,  24],
         [100, 100, 100, 100, 100, 100, 100],
         [ 61,  81,  98,  11,  31,  69,  91],
         [ 93,  94,  59,   6,  54,  18,   3],
         [ 94,  88,   0,  59,  41,  41,  27],
         [100, 100, 100, 100, 100, 100, 100]],

        [[100, 100, 100, 100, 100, 100, 100],
         [ 28,  37,   2,  49,  12,  11,  47],
         [ 57,  29,  79,  19,  95,  84,   7],
         [ 37,  52,  57,  61,  69,  52,  25],
         [ 73,   2,  20,  37,  25,  32,   9],
         [100, 100, 100, 100, 100, 100, 100],
         [ 45,  60,  81,  97,  81,  97,  46],
         [  5,  26,  84,  49,  25,  11,   3],
         [  7,  39,  77,  77,   1,  81,  10],
         [100, 100, 100, 100, 100, 100, 100]],

        [[100, 100, 100, 100, 100, 100, 100],
         [ 95,  68,   4,  81, 

### 三、维度变换

维度变换相关的主要函数有torch.reshape, torch.squeeze, 

torch.unsqueeze, torch.transpose

torch.reshape可以改变张量形状

torch.squeeze可以减少维度

torch.unsqueeze可以增加维度

torch.transpose可以交换维度

In [36]:
# 张量的view方法有时会调用失败, 可以使用reshape方法
torch.manual_seed(0)
minval, maxval = 0, 255
a = (minval + (maxval-minval)*torch.rand([1, 3, 3, 2])).int()
print(a.shape)
print(a)

torch.Size([1, 3, 3, 2])
tensor([[[[126, 195],
          [ 22,  33],
          [ 78, 161]],

         [[124, 228],
          [116, 161],
          [ 88, 102]],

         [[  5,  43],
          [ 74, 132],
          [177, 204]]]], dtype=torch.int32)


In [37]:
# 改成(3, 6)形状的张量
b = a.view([3, 6])
print(b.shape)
print(b)

torch.Size([3, 6])
tensor([[126, 195,  22,  33,  78, 161],
        [124, 228, 116, 161,  88, 102],
        [  5,  43,  74, 132, 177, 204]], dtype=torch.int32)


In [38]:
c = torch.reshape(b, [1, 3, 3, 2])
print(c)

tensor([[[[126, 195],
          [ 22,  33],
          [ 78, 161]],

         [[124, 228],
          [116, 161],
          [ 88, 102]],

         [[  5,  43],
          [ 74, 132],
          [177, 204]]]], dtype=torch.int32)


如果张量在某个维度只有一个元素, 利用torch.squeeze可以消除这个维度。

torch.unsqueeze的作用和torch.squeeze的作用相反

In [39]:
a = torch.tensor([[1.0, 2.0]])
s = torch.squeeze(a)
print(a)
print(s)
print(a.shape)
print(s.shape)

tensor([[1., 2.]])
tensor([1., 2.])
torch.Size([1, 2])
torch.Size([2])


torch.transpose可以交换张量的维度, torch.transpose常用于图片储存格式的变换上。

如果是二维的矩阵, 通常会调用矩阵的转置方法matrix.t(), 等价于torch.tranpose(matrix, 0, 1)

In [40]:
minval = 0
maxval = 255
# batch, height, width, channel
data = torch.floor(minval + (maxval-minval)*torch.rand([100, 256, 256, 4])).int()
print(data.shape)

torch.Size([100, 256, 256, 4])


In [46]:
# 交换成Pytorch默认的图片格式 batch, channel, height, width
data_t = torch.transpose(torch.transpose(data, 1, 3), 2, 3)
print(data_t.shape)

torch.Size([100, 4, 256, 256])


In [47]:
matrix = torch.tensor([[1,2,3],[4,5,6]])
print(matrix)
print(matrix.t()) #等价于torch.transpose(matrix,0,1)

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])


### 四、合并分割
可以使用torch.cat方法和torch.stack方法将多个张量合并, 可以用torch.split方法把一个张量分割成多个张量。

torch.cat和torch.stack有略微区别, 前者是连接, 不会增加维度, 而后者是堆叠, 会增加维度。

In [49]:
a = torch.tensor([[1., 2], [3, 4]])
b = torch.tensor([[5., 6], [7, 8]])
c = torch.tensor([[9., 10], [11, 12]])

abc_cat = torch.cat([a, b, c], dim=0)
print(abc_cat.shape)
print(abc_cat)

torch.Size([6, 2])
tensor([[ 1.,  2.],
        [ 3.,  4.],
        [ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.],
        [11., 12.]])


In [50]:
abc_stack = torch.stack([a, b, c], axis=0)
print(abc_stack.shape)
print(abc_stack)

torch.Size([3, 2, 2])
tensor([[[ 1.,  2.],
         [ 3.,  4.]],

        [[ 5.,  6.],
         [ 7.,  8.]],

        [[ 9., 10.],
         [11., 12.]]])


torch.split是torch.cat的逆运算, 可以指定分割平均数, 也可以指定每份记录数量

In [51]:
a, b, c = torch.split(abc_cat, split_size_or_sections=2, dim=0)
print(a)
print(b)
print(c)

tensor([[1., 2.],
        [3., 4.]])
tensor([[5., 6.],
        [7., 8.]])
tensor([[ 9., 10.],
        [11., 12.]])


In [52]:
print(abc_cat)
p,q,r = torch.split(abc_cat,split_size_or_sections =[4,1,1],dim = 0) #每份分别为[4,1,1]
print(p)
print(q)
print(r)

tensor([[ 1.,  2.],
        [ 3.,  4.],
        [ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.],
        [11., 12.]])
tensor([[1., 2.],
        [3., 4.],
        [5., 6.],
        [7., 8.]])
tensor([[ 9., 10.]])
tensor([[11., 12.]])
